![Astrofisica Computacional](../logo.PNG)

---
## 25. Validación cruzada del Árbol de Decisión 

Eduard Larrañaga (ealarranaga@unal.edu.co)

---

### Resumen

En este cuaderno se introducirá el método de validación cruzada de los árboles de decisión.

---

Hasta este momento, se ha utilizado la mediana de los residuos para medir la precisión de la predicción. Este método básico, en el que se divide el conjunto de datos en subconjuntos de entrenamiento y prueba, es llamado **hold-out validation** (validación de retención) y el resultado de la precisión dependerá de como se dividen los subconjuntos.

Ahora, se introducirá un mejor método de validación denominado **k-fold cross-validation** (validación cruzada de k-combinaciones). Este es similar a la validación hold-out, excepto porque los datos originales se dividiran en k-subconjuntos y se entrenara el modelo k-veces, utilizando diferentes combinaciones de los subconjuntos y registrando la precisión en cada ocasión (i.e. se realizará una validación hold-out en k-ocasiones). 

En la práctica, en cada ocasión se utiliza una combinación diferente de k-1 subconjuntos para entrenar el modelo y el subconjunto restante se utiliza para la prueba del modelo. Luego, se toma el promedio de las precisiones de las k-mediciones para obtener la precisión global del modelo.

De nuevo utilizaremos el conjunto de datos fotométricos de galaxias, pero en esta ocasión introduciremos un aspecto adicional al estimar la precisión cuando el modelo se aplica a Quasi-Stellar Objects (QSOs) comparado con otras galaxias. Como es bien conocido, los QSOs son galaxias que poseen un núcleo activo (AGN), que hacen que la galxia sea más brillante y poor lo tanto, detectable con los instrumentos del SDSS con corrimientos al rojo mas altos. 

### Cargando los Datos

Igual que antes, se utiliza el archivo `'sdss_galaxy_colors.npy`. 


In [ ]:
import numpy as np

data = np.load('sdss_galaxy_colors.npy')
data

array([(19.84132, 19.52656, 19.46946, 19.17955, 19.10763, b'QSO', 0.539301  , 6.543622e-05),
       (19.86318, 18.66298, 17.84272, 17.38978, 17.14313, b'GALAXY', 0.1645703 , 1.186625e-05),
       (19.97362, 18.31421, 17.47922, 17.0744 , 16.76174, b'GALAXY', 0.04190006, 2.183788e-05),
       ...,
       (19.82667, 18.10038, 17.16133, 16.5796 , 16.19755, b'GALAXY', 0.0784592 , 2.159406e-05),
       (19.98672, 19.75385, 19.5713 , 19.27739, 19.25895, b'QSO', 1.567295  , 4.505933e-04),
       (18.00024, 17.80957, 17.77302, 17.72663, 17.7264 , b'QSO', 0.4749449 , 6.203324e-05)],
      dtype=[('u', '<f8'), ('g', '<f8'), ('r', '<f8'), ('i', '<f8'), ('z', '<f8'), ('spec_class', 'S6'), ('redshift', '<f8'), ('redshift_err', '<f8')])

Las características contenidas en el archivo son

| dtype | Feature|
|:-:|:-:|
|`u` |u band filter|
|`g` |g band filter|
|`r` |r band filter|
|`i` |i band filter|
|`z` |z band filter|
|`spec_class` |spectral class|
|`redshift` |redshift|
|`redshift_err` |redshift error|


El número de muestras (galaxias) en el archivo es

In [ ]:
n = data.size
n

50000

El primer paso es definir los conjuntos de entrada 'features' (indices de color) y 'targets' (corrimiento al rojo)

In [ ]:
# Function returning the 4 color indices and the redshifts

features, targets = ...


### K-Fold

Para hacer la división del conjunto en los k-subconjuntos se utilizará la función [sklearn.model_selection.KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html). La división se realiza definiendo un objeto iterable mediante el comando

```
kf = KFold(n_splits=k, shuffle=True)
```

dodne el argumento `n_splits=k` especifica el número de subconjuntos que se utilizarán y el argumento  `shuffle` es puesto en Falso por defecto, pero es una buena práctica activarlo para que se eligan aleatoriamente los elementos de cada subconjunto (con ello se evita introducir una tendencia o parcialidad debido al orden en el que se presentan los datos dentro del archivo). 

Se utilizará la función `KFold` con un valor inicial de **k=5** y el árbol de decisión con una profundidad de   `max_depth=19` (de acuerdo con el resultado anterior para evitar el sobre-ajuste).

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True)
dec_tree = DecisionTreeRegressor(max_depth=19)


El método `.split()` se aplica al conjunto de características para generar los conjuntos de entrenamiento y prueba. Nótese que este método define solo el conjunto de índices correspondeintes a cada subconjunto, pero no el subconjunto como tal. Por esta razón, debemos tomar estos índices y definir los conjuntos adecuadamente para luego proceder a entrenar el modelo y evaluar su predicción mediante la mediana de los residuos.

Este proceso completo debe repetirse para cada una de las k-iteraciones. Para ello implementamos un loop  `for`,



In [ ]:
# declare an array for predicted redshifts from each iteration
all_predictions = np.zeros_like(targets)

for train_indices, test_indices in kf.split(features):
  train_features, test_features = features[train_indices], features[test_indices]
  train_targets, test_targets = targets[train_indices], targets[test_indices]

  # Train the decision tree
  dec_tree.fit(train_features, train_targets)
  
  # Predict using the model
  predictions = dec_tree.predict(test_features)

  # put the predicted values in the all_predictions array defined above
  all_predictions[test_indices] = predictions



# Evaluate the model using the median of differences of all_predictions
eval_dec_tree = ...  
eval_dec_tree

Una vez que se entrena el modelo y se calculan las predicciones , se pueden compara gráficamente las predicciones y los datos 'target'


In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

# plot the results to see how well our model looks
plt.figure()
plt.scatter(targets, all_predictions, s=0.4)
plt.xlim((0, targets.max()))
plt.ylim((0, predictions.max()))
plt.xlabel('Measured Redshift')
plt.ylabel('Predicted Redshift')
plt.show()


El resultado debe lucir como la siguiente figura:

<center>
<img src="https://groklearning-cdn.com/modules/SjroKib6Hs5Fqxq53Vxme9/predicted_v_measured.png" width=450>
</center>


Nótese que en este gráfico de predicciones contra mediciones se observa un buen comportamiento para muchas galaxias, pero también exiten muchos puntos fuera de la tendencia (outliers).

---
### Clase Espectral 

La característica 'spec_class' en el conjunto de datos involucra dos posibles valores b'GALAXY' y b'QSO', que identifican a las galaxias y a los Quasi-Stellar Objects (QSOs), respectivamente.

**1. Defina una función que clasificque las muestras de acuerdo con el 'spec_class'.**

**2. Cuantas galaxias y cuantos QSOs existen en el archivo?**


**3. Calcular la mediana de los residuos para galaxias y QSOs. Cuáles son los valores máximos de estas diferencias para los dos tipos de objetos?**


Las galaxias no son tan brillantes como los QSOs y por ello son muy débiles para ser detectados por el SDSS si poseen un redshift superior a 0.4. Esto crea una parcialidad en las mediciones.

**4. Realice un gráfico con la mediana de los residuos vs. Redshift medido para todos los objetos del conjunto y utilice un color para identificar galaxias y otro color para los QSOs.**


**5. Realice unnuevo gráfico con las predicciones vs mediciones de corrimiento al rojo, identificando el tipo de objeto con colorse diferentes. El resultado debe ser similar a este:**

<center>
<img src="https://groklearning-cdn.com/modules/ovFSymwFkqBPAcjnbSUxLG/predicted_actual_qso.png" width=450>
</center>

